In [247]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json

In [273]:
movie_data = pd.read_csv("movie_data/movies_metadata.csv")
movie_data.head()

/Users/alec/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [274]:
movie_data.shape

(45466, 24)

In [275]:
movie_data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [276]:
movie_data.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [277]:
# convert budget col to number

# get rid of rows that contain letters first 
bad_budget_rows = movie_data["budget"].str.isnumeric()
movie_data = movie_data[bad_budget_rows]
movie_data["budget"] = movie_data["budget"].astype(np.float64)
movie_data = movie_data[(movie_data["revenue"] != 0) & (movie_data["budget"] != 0) & (movie_data["runtime"] !=0)].reset_index(drop=True)

In [435]:
# columns that are already cleaned
movie_df = movie_data[["id", "revenue", "vote_average", "title", "budget", "runtime", "original_language"]].copy()
movie_df.shape

(5370, 7)

In [436]:
# JSON cols 

# belongs to collection -> boolean
# genres -> top 1
# production companies -> top 1
# production countries -> top 1 
# release date -> extract month and year 

In [437]:
# Franchise
movie_df["is_franchise"] = ~movie_data["belongs_to_collection"].isna()

In [438]:
# genres 
def get_json_val(x):
    x_dict = eval(x)
    return np.nan if len(x_dict) == 0 else x_dict[0]["name"]

movie_df["genre"] = movie_data["genres"].apply(get_json_val)
movie_df.dropna(inplace=True)
movie_df.shape

(5358, 9)

In [439]:
# production company 
movie_df["production_company"] = movie_data["production_companies"].apply(get_json_val)
movie_df.dropna(inplace=True)
movie_df.shape

(5193, 9)

In [441]:
# country
movie_df["country"] = movie_data["production_countries"].apply(get_json_val)
movie_df.dropna(inplace=True)
movie_df.shape

(5181, 10)

In [450]:
# release date 
dates = pd.to_datetime(movie_data["release_date"])
movie_df["release_month"] = dates.dt.month
movie_df["release_year"] = dates.dt.year
movie_df.dropna(inplace=True)
movie_df.head()

,id,revenue,vote_average,title,budget,runtime,original_language,is_franchise,genre,country,release_month,release_year
0,862,373554033.0,7.7,Toy Story,30000000.0,81.0,en,True,Pixar Animation Studios,United States of America,10,1995
1,8844,262797249.0,6.9,Jumanji,65000000.0,104.0,en,False,TriStar Pictures,United States of America,12,1995
2,31357,81452156.0,6.1,Waiting to Exhale,16000000.0,127.0,en,False,Twentieth Century Fox Film Corporation,United States of America,12,1995
3,949,187436818.0,7.7,Heat,60000000.0,170.0,en,False,Regency Enterprises,United States of America,12,1995
4,9091,64350171.0,5.5,Sudden Death,35000000.0,106.0,en,False,Universal Pictures,United States of America,12,1995
